# Create a subset of Pascal Part Person dataset

In [9]:
from grcore.coco import ann, save_cocodict
from grcore.coco.dataset import CocoDataset
import numpy as np
import copy
import shutil
from pathlib import Path
from rich.jupyter import print

In [10]:
%pprint

Pretty printing has been turned OFF


In [11]:
imagecount = 100

In [12]:
base_path = Path("/data/dataset/pascal/pascalparts_person")
wpath = base_path / "annotations/val_wholes.json"
ppath = base_path / "annotations/val_parts.json"


In [13]:
coco = CocoDataset(wpath, ppath, from_gt=True)

inforecord = copy.deepcopy(coco.gt.dataset["info"])
licrecords = copy.deepcopy(coco.gt.dataset["licenses"])
catrecords = copy.deepcopy(coco.gt.dataset["categories"])

loading annotations into memory...
Loading json: /data/dataset/pascal/pascalparts_person/annotations/val_wholes.json
Done 0.02s
creating index...
index created!
loading annotations into memory...
Loading json: /data/dataset/pascal/pascalparts_person/annotations/val_parts.json
Done 0.11s
creating index...
index created!


In [14]:
imgids = np.random.choice(coco.imgids, imagecount, replace=False).tolist()
print(len(imgids))
imgrecords = copy.deepcopy(coco.load_imginfos(imgids))

imgmap = {}
for id_, imgrecord in enumerate(imgrecords, 1):
    imgmap[imgrecord["id"]] = id_
    imgrecord["id"] = id_

100

In [15]:
annids = coco.get_annids(imgids=imgids)
wholes_annrecords = copy.deepcopy(coco.load_anninfos(annids))
for id_, annrecord in enumerate(wholes_annrecords, 1):
    annrecord["id"] = id_
    annrecord["image_id"] = imgmap[annrecord["image_id"]]

In [16]:
wholesdset = ann.build_dset_info(inforecord, licrecords, catrecords, imgrecords, wholes_annrecords)
wholepath = coco.root_path / "annotations" / "val_wholes_subset.json"
save_cocodict(wholesdset, wholepath)

Saving json: /data/dataset/pascal/pascalparts_person/annotations/val_wholes_subset.json


In [17]:
annids = coco.get_annids(imgids=imgids, from_gt=False)
parts_annrecords = coco.load_anninfos(annids, from_gt=False)
for id_, annrecord in enumerate(parts_annrecords, 1):
    annrecord["id"] = id_
    annrecord["image_id"] = imgmap[annrecord["image_id"]]

In [18]:
partsdset = ann.build_dset_info(inforecord, licrecords, catrecords, imgrecords, parts_annrecords)
valpartspath = coco.root_path / "annotations" / "val_parts_subset.json"
save_cocodict(partsdset, valpartspath)

Saving json: /data/dataset/pascal/pascalparts_person/annotations/val_parts_subset.json


## Save val images

for imgrecord in imgrecords:
    src = coco.root_path / "val_noocln" / imgrecord["file_name"]
    dst = coco.root_path / imgrecord["coco_url"]
    shutil.copy(src, dst)